In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/amazon-delivery-dataset/amazon_delivery.csv


In [2]:
df = pd.read_csv("/kaggle/input/amazon-delivery-dataset/amazon_delivery.csv")
df.head()

,Order_ID,Agent_Age,Agent_Rating,Store_Latitude,Store_Longitude,Drop_Latitude,Drop_Longitude,Order_Date,Order_Time,Pickup_Time,Weather,Traffic,Vehicle,Area,Delivery_Time,Category
0,ialx566343618,37,4.9,22.745049,75.892471,22.765049,75.912471,2022-03-19,11:30:00,11:45:00,Sunny,High,motorcycle,Urban,120,Clothing
1,akqg208421122,34,4.5,12.913041,77.683237,13.043041,77.813237,2022-03-25,19:45:00,19:50:00,Stormy,Jam,scooter,Metropolitian,165,Electronics
2,njpu434582536,23,4.4,12.914264,77.678400,12.924264,77.688400,2022-03-19,08:30:00,08:45:00,Sandstorms,Low,motorcycle,Urban,130,Sports
3,rjto796129700,38,4.7,11.003669,76.976494,11.053669,77.026494,2022-04-05,18:00:00,18:10:00,Sunny,Medium,motorcycle,Metropolitian,105,Cosmetics
4,zguw716275638,32,4.6,12.972793,80.249982,13.012793,80.289982,2022-03-26,13:30:00,13:45:00,Cloudy,High,scooter,Metropolitian,150,Toys


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43739 entries, 0 to 43738
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Order_ID         43739 non-null  object 
 1   Agent_Age        43739 non-null  int64  
 2   Agent_Rating     43685 non-null  float64
 3   Store_Latitude   43739 non-null  float64
 4   Store_Longitude  43739 non-null  float64
 5   Drop_Latitude    43739 non-null  float64
 6   Drop_Longitude   43739 non-null  float64
 7   Order_Date       43739 non-null  object 
 8   Order_Time       43739 non-null  object 
 9   Pickup_Time      43739 non-null  object 
 10  Weather          43648 non-null  object 
 11  Traffic          43739 non-null  object 
 12  Vehicle          43739 non-null  object 
 13  Area             43739 non-null  object 
 14  Delivery_Time    43739 non-null  int64  
 15  Category         43739 non-null  object 
dtypes: float64(5), int64(2), object(9)
memory usage: 5.3+ MB


In [4]:
df.describe()

,Agent_Age,Agent_Rating,Store_Latitude,Store_Longitude,Drop_Latitude,Drop_Longitude,Delivery_Time
count,43739.000000,43685.000000,43739.000000,43739.000000,43739.000000,43739.000000,43739.000000
mean,29.567137,4.633780,17.210960,70.661177,17.459031,70.821842,124.905645
std,5.815155,0.334716,7.764225,21.475005,7.342950,21.153148,51.915451
min,15.000000,1.000000,-30.902872,-88.366217,0.010000,0.010000,10.000000
25%,25.000000,4.500000,12.933298,73.170283,12.985996,73.280000,90.000000
50%,30.000000,4.700000,18.551440,75.898497,18.633626,76.002574,125.000000
75%,35.000000,4.900000,22.732225,78.045359,22.785049,78.104095,160.000000
max,50.000000,6.000000,30.914057,88.433452,31.054057,88.563452,270.000000


In [5]:
df.isnull().sum()

Order_ID            0
Agent_Age           0
Agent_Rating       54
Store_Latitude      0
Store_Longitude     0
Drop_Latitude       0
Drop_Longitude      0
Order_Date          0
Order_Time          0
Pickup_Time         0
Weather            91
Traffic             0
Vehicle             0
Area                0
Delivery_Time       0
Category            0
dtype: int64

In [6]:
df.duplicated().sum()

0

In [7]:
# Handling Missing data

median_rating = df['Agent_Rating'].median()
df['Agent_Rating'].fillna(median_rating)

mode_weather = df['Weather'].mode()[0]
df['Weather'].fillna(mode_weather)

df.isnull().sum()

Order_ID            0
Agent_Age           0
Agent_Rating       54
Store_Latitude      0
Store_Longitude     0
Drop_Latitude       0
Drop_Longitude      0
Order_Date          0
Order_Time          0
Pickup_Time         0
Weather            91
Traffic             0
Vehicle             0
Area                0
Delivery_Time       0
Category            0
dtype: int64

In [8]:
df['Order_Time'].isnull().sum()

0

In [9]:
df['Order_Time'].value_counts().head(10)


Order_Time
21:55:00    460
17:55:00    453
20:00:00    446
22:20:00    446
21:35:00    445
19:50:00    444
21:20:00    437
21:15:00    437
18:10:00    435
22:45:00    435
Name: count, dtype: int64

In [10]:
# Standardize Order_Time values
df['Order_Time'] = df['Order_Time'].astype(str).str.strip().str.lower()

# Remove rows with missing, nan, or empty
valid_times_mask = (~df['Order_Time'].isin(['nan', '', 'none'])) & (df['Order_Time'].notna())
df_cleaned = df[valid_times_mask].copy()


In [11]:
df_cleaned['Order_DateTime'] = pd.to_datetime(df_cleaned['Order_Date'] + ' ' + df_cleaned['Order_Time'])
df_cleaned.head()

,Order_ID,Agent_Age,Agent_Rating,Store_Latitude,Store_Longitude,Drop_Latitude,Drop_Longitude,Order_Date,Order_Time,Pickup_Time,Weather,Traffic,Vehicle,Area,Delivery_Time,Category,Order_DateTime
0,ialx566343618,37,4.9,22.745049,75.892471,22.765049,75.912471,2022-03-19,11:30:00,11:45:00,Sunny,High,motorcycle,Urban,120,Clothing,2022-03-19 11:30:00
1,akqg208421122,34,4.5,12.913041,77.683237,13.043041,77.813237,2022-03-25,19:45:00,19:50:00,Stormy,Jam,scooter,Metropolitian,165,Electronics,2022-03-25 19:45:00
2,njpu434582536,23,4.4,12.914264,77.678400,12.924264,77.688400,2022-03-19,08:30:00,08:45:00,Sandstorms,Low,motorcycle,Urban,130,Sports,2022-03-19 08:30:00
3,rjto796129700,38,4.7,11.003669,76.976494,11.053669,77.026494,2022-04-05,18:00:00,18:10:00,Sunny,Medium,motorcycle,Metropolitian,105,Cosmetics,2022-04-05 18:00:00
4,zguw716275638,32,4.6,12.972793,80.249982,13.012793,80.289982,2022-03-26,13:30:00,13:45:00,Cloudy,High,scooter,Metropolitian,150,Toys,2022-03-26 13:30:00


In [12]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 43648 entries, 0 to 43738
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Order_ID         43648 non-null  object        
 1   Agent_Age        43648 non-null  int64         
 2   Agent_Rating     43594 non-null  float64       
 3   Store_Latitude   43648 non-null  float64       
 4   Store_Longitude  43648 non-null  float64       
 5   Drop_Latitude    43648 non-null  float64       
 6   Drop_Longitude   43648 non-null  float64       
 7   Order_Date       43648 non-null  object        
 8   Order_Time       43648 non-null  object        
 9   Pickup_Time      43648 non-null  object        
 10  Weather          43648 non-null  object        
 11  Traffic          43648 non-null  object        
 12  Vehicle          43648 non-null  object        
 13  Area             43648 non-null  object        
 14  Delivery_Time    43648 non-null  int64     

In [13]:
df_cleaned['order_hour'] = df_cleaned['Order_DateTime'].dt.hour
df_cleaned['order_dayofweek'] = df_cleaned['Order_DateTime'].dt.dayofweek  
df_cleaned['is_weekend'] = df_cleaned['order_dayofweek'].apply(lambda x: 1 if x >= 5 else 0)


In [14]:
# Calculate distance using Haversine formula
from math import radians, cos, sin, asin, sqrt

def haversine(lat1, lon1, lat2, lon2):
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1 
    dlon = lon2 - lon1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371  # Earth's radius in kilometers
    return c * r

# Apply to all rows
df_cleaned['distance_km'] = df_cleaned.apply(
    lambda row: haversine(row['Store_Latitude'], row['Store_Longitude'],
                          row['Drop_Latitude'], row['Drop_Longitude']), axis=1)


In [15]:
# Create Binary target
delay_threshold = 150  # Threshold in minutes
df_cleaned['Is_Delayed'] = df_cleaned['Delivery_Time'].apply(lambda x: 1 if x > delay_threshold else 0)


In [16]:
df_cleaned[['Order_DateTime', 'order_hour', 'order_dayofweek', 'is_weekend', 'distance_km', 'Is_Delayed']].head()


,Order_DateTime,order_hour,order_dayofweek,is_weekend,distance_km,Is_Delayed
0,2022-03-19 11:30:00,11,5,1,3.025149,0
1,2022-03-25 19:45:00,19,4,0,20.183530,1
2,2022-03-19 08:30:00,8,5,1,1.552758,0
3,2022-04-05 18:00:00,18,1,0,7.790401,0
4,2022-03-26 13:30:00,13,5,1,6.210138,0


## Classification 
# Predict whether a Delivery is Delayed?

In [17]:
# Define Features and target

# Feature groups
numerical_features = ['Agent_Age', 'Agent_Rating', 'order_hour', 'order_dayofweek', 'is_weekend', 'distance_km']
categorical_features = ['Weather', 'Traffic', 'Vehicle', 'Area', 'Category']
target = 'Is_Delayed'

# Input matrix and target vector
X = df_cleaned[numerical_features + categorical_features]
y = df_cleaned[target]


In [18]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

# Define transformers for numeric and categorical columns
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median'))
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combine preprocessing
preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, numerical_features),
    ('cat', categorical_transformer, categorical_features)
])


In [19]:
# Split dataset:

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y)


In [20]:
# Train with Logistic regression

from sklearn.linear_model import LogisticRegression

# Create full pipeline
logreg_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(max_iter=1000))
])

# Train the model
logreg_pipeline.fit(X_train, y_train)


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  ['Agent_Age', 'Agent_Rating',
                                                   'order_hour',
                                                   'order_dayofweek',
                                                   'is_weekend',
                                                   'distance_km']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Weather', 'Traffic',
                                                   'Vehicle', 'Area',
                                                   'Category'])])),
                ('classifier', LogisticRegression(max_iter=1000))])

In [21]:
# Evaluate the model
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

# Predict
y_pred = logreg_pipeline.predict(X_test)
y_proba = logreg_pipeline.predict_proba(X_test)[:, 1]

# Evaluation
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("ROC AUC Score:\n", roc_auc_score(y_test, y_proba))


Classification Report:
               precision    recall  f1-score   support

           0       0.89      0.93      0.91      6294
           1       0.80      0.71      0.75      2436

    accuracy                           0.87      8730
   macro avg       0.85      0.82      0.83      8730
weighted avg       0.87      0.87      0.87      8730

Confusion Matrix:
 [[5868  426]
 [ 713 1723]]
ROC AUC Score:
 0.9304873982728097


In [22]:
import joblib

# Save model pipeline to disk
joblib.dump(logreg_pipeline, 'logistics_delay_classifier.pkl')


['logistics_delay_classifier.pkl']

## Regression Model
### Predict Delivery Time

In [23]:
# Regression model
numerical_features = ['Agent_Age', 'Agent_Rating', 'order_hour', 'order_dayofweek', 'is_weekend', 'distance_km']
categorical_features = ['Weather', 'Traffic', 'Vehicle', 'Area', 'Category']
target = 'Delivery_Time'


In [24]:
# Preprocessing Pipelines
# For numerical columns: fill missing values with median
numeric_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='median'))
])

# For categorical columns: fill with most frequent, then one-hot encode
categorical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


In [25]:
preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, numerical_features),
    ('cat', categorical_transformer, categorical_features)
])


In [26]:
# Train Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [27]:
# Regression pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

regressor = Pipeline([
    ('preprocessor', preprocessor),
    ('model', RandomForestRegressor(n_estimators=100, random_state=42))
])


In [28]:
regressor.fit(X_train, y_train)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  ['Agent_Age', 'Agent_Rating',
                                                   'order_hour',
                                                   'order_dayofweek',
                                                   'is_weekend',
                                                   'distance_km']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Weather', 'Traffic',
                                                   'Vehicle', 'Area',
                                                   'Category'])])),
                ('model', RandomForestRegressor(random_state=42))])

In [29]:
# Predict values
y_pred = regressor.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print('MAE:' , mae)
print("RMSE: ", rmse)
print("R: ", r2)


MAE: 0.09984421534936999
RMSE:  0.23229952620269742
R:  0.7378119311487951


In [30]:
joblib.dump(regressor, 'delivery_time_regressor.pkl')


['delivery_time_regressor.pkl']